In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
#from evcouplings.align import Alignment, map_matrix, ALPHABET_RNA, ALPHABET_PROTEIN
ALPHABET_PROTEIN = '-ACDEFGHIKLMNPQRSTVWY'
from collections import OrderedDict
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

from itertools import combinations, product
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from joblib import Parallel, delayed, parallel_backend
from pathlib import Path
from Bio import SeqIO

import scipy.stats
from scipy.stats import spearmanr
from potts import Potts
from potts.mcmc import GWGCategoricalSampler, ReplayBuffer, CategoricalGibbsSampler, OneHotCategorical, CategoricalMetropolistHastingsSampler, PottsGWGCategoricalSampler, UniformCategoricalSampler, PottsGWGHardWallCategoricalSampler
from potts.structure import get_single_double_mutants, esm_inv_nll_from_encoder, esm_inv_batch_encoding, potts_from_nlls
import pandas as pd
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

/data/mambaforge/envs/pytorch_hunter/lib/python3.10/site-packages/esm/pretrained.py:215: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


In [3]:
torch.cuda.set_device(0)

In [4]:
def encode(seqs, alphabet=ALPHABET_PROTEIN, verbose=True):
    '''
    Go from letters to numbers
    '''
    aa_to_i = OrderedDict((aa, i) for i, aa in enumerate( alphabet ))
    if verbose:
        seq_iter = tqdm(seqs)
    else:
        seq_iter = seqs
    X = torch.tensor([[aa_to_i[x] for x in seq] 
                      for seq in seq_iter])
    return X, aa_to_i

In [13]:
def train_sgd(model, train_X, train_w, lam, bs=1000, lr=1e-1, device='cuda', n_epoch=10,
              mut_seqs_onehot=None, y_dms=None, contact_map=None):
    L, A = model.L, model.A
    wrs = torch.utils.data.WeightedRandomSampler(train_w / train_w.sum(), len(train_X), replacement=True)
    bs = min(len(train_X), bs)
    train_dl = DataLoader(TensorDataset(train_X, train_w), batch_size=bs, drop_last=True, sampler=wrs)
    model.to(device)
    optim = torch.optim.SGD(model.parameters(), lr=lr)
    epoch_iter = tqdm(range(n_epoch), total=n_epoch, position=0)
    num_iters = 0
    spearman = 0

    if not contact_map is None:
        contact_map = torch.tensor(contact_map).unsqueeze(-1).unsqueeze(-1).cuda()
    for epoch in epoch_iter:
        epoch_l = 0.0
        model.train()
        total_train_pl = []
        total_w = []
        for X, w in train_dl:
            X, w = X.to(device), w.to(device)
            optim.zero_grad()
            mask = X.argmax(dim=-1) != 0
            mask_w = mask.sum(dim=-1) / mask.shape[-1]
            pl = model.pseudolikelihood(X, mask=mask) * mask_w
            #pl = model.pseudolikelihood(X, mask=None)
            loss = -(w * pl).sum() / w.sum()
            epoch_l += (-(w * pl).sum()).item()
            # regularization
            W = model.W.weight
            reg_loss = (model.h ** 2).sum()
            reg_loss += ((W ** 2).sum() ) * (model.L - 1) * (model.A - 1)
            loss += lam * reg_loss 
            loss.backward()
            optim.step()

            num_iters += 1
            #epoch_iter.set_postfix(spearman=spearman, num_iters=num_iters, loss=loss.item(), reg_loss=lam*reg_loss.item())
        if not (mut_seqs_onehot is None):
            with torch.no_grad():
                bs = 1000
                all_preds = []
                for i in range(np.ceil(len(mut_seqs_onehot) / bs).astype(int)):
                    prior_pred = -model(mut_seqs_onehot[i*bs:(i+1)*bs].to(device)).cpu().numpy()
                    all_preds.append(prior_pred)
                all_preds = np.concatenate(all_preds)
                spearman = spearmanr(all_preds, y_dms).correlation
                epoch_iter.set_postfix(spearman=spearman, num_iters=num_iters, loss=loss.item(), mem=torch.cuda.memory_allocated() , epoch_l=epoch_l)
    model.eval()
    return model


In [7]:
df = pd.read_csv('/home/hunter/projects/recombination/ProteinGym/Tranception/proteingym/Detailed_performance_files/Substitutions/Spearman/all_models_substitutions_Spearman_DMS_level.csv')
df = df.rename(columns={df.columns[0]: 'Dataset'})
dataset = 'BLAT_ECOLX_Stiffler_2015'

# get beta-lactamase MSA
msa_fn = Path('/home/hunter/projects/recombination/ProteinGym/MSA_files/BLAT_ECOLX_full_11-26-2021_b02.a2m')
# get beta-lactamase DMS data
mut_fn = Path('/home/hunter/projects/recombination/ProteinGym/ProteinGym_substitutions/BLAT_ECOLX_Stiffler_2015.csv')
# get beta-lactamase MSA sequence weights
weight_fn = Path('/home/hunter/projects/recombination/ProteinGym/substitutions_MSAs_all_positions/BLAT_ECOLX_theta_0.2.npy')
# get ProteinGym results
res_fn = Path('/home/hunter/projects/recombination/ProteinGym/substitutions/BLAT_ECOLX_Stiffler_2015.csv')


mut_df = pd.read_csv(mut_fn)
res_df = pd.read_csv(res_fn)


y_bin = mut_df.DMS_score_bin.to_numpy()
ev_spearman = scipy.stats.spearmanr(res_df.EVmutation, res_df.DMS_score).correlation
print()
print(f"EVmutation Spearman: {ev_spearman:.3f}", )


EVmutation Spearman: 0.707


In [8]:
aa_to_i = OrderedDict((aa, i) for i, aa in enumerate( ALPHABET_PROTEIN ))
y_dms = res_df.DMS_score.to_numpy()
mut_seqs = mut_df.mutated_sequence.map(lambda x: [aa_to_i[x[i]] for i in range(len(x))]).to_list()
mut_seqs = torch.tensor(mut_seqs)
mut_seqs_onehot = torch.nn.functional.one_hot(mut_seqs, num_classes=len(ALPHABET_PROTEIN)).to(torch.float)


In [9]:
'''
CREATE TRAINING DATA
'''
def check_sequence(s, alphabet=ALPHABET_PROTEIN):
    for aa in s:
        if aa not in ALPHABET_PROTEIN:
            return False
    return True

msa_sequences = [str(x.seq) for x in SeqIO.parse(msa_fn, 'fasta')]
msa_sequences = [s.replace(".", "-").upper() for s in msa_sequences]
wt_seq = msa_sequences[0]
columns_to_keep = [i for i in range(len(wt_seq))]
msa_sequences = [[s[i] for i in columns_to_keep] for s in msa_sequences]
#msa_sequences = [[s[i].upper() for i in columns_to_keep] for s in msa_sequences]
#msa_sequences = [[aa.replace(".", "-") for aa in s] for s in msa_sequences]
msa_sequences = np.asarray(msa_sequences)
msa_sequences = [s for s in msa_sequences if check_sequence(s)]
msa_sequences = np.asarray(msa_sequences)
if False:
    threshold_sequence_frac_gaps=0.5
    threshold_focus_cols_frac_gaps=0.3
    gap_array = np.asarray([[1 if aa == "-" else 0 for aa in s] for s in msa_sequences])
    msa_sequences = msa_sequences[~(gap_array.mean(axis=1) > threshold_sequence_frac_gaps)]
    msa_sequences = msa_sequences[:, ~(gap_array.mean(axis=0) > threshold_focus_cols_frac_gaps)]

seqs_enc, aa_to_i = encode(msa_sequences)
i_to_a = {i:aa for i, aa in enumerate(ALPHABET_PROTEIN)}
weights = np.load(weight_fn)
assert weights.shape[0] == len(msa_sequences)
seqs_onehot = torch.nn.functional.one_hot(seqs_enc, num_classes=len(ALPHABET_PROTEIN)).to(torch.float)
train_ds = TensorDataset(seqs_onehot, torch.tensor(weights))

  0%|                                                                                                                                                             | 0/208923 [00:00<?, ?it/s]

  1%|█▉                                                                                                                                             | 2904/208923 [00:00<00:07, 29034.49it/s]

  3%|███▉                                                                                                                                           | 5808/208923 [00:00<00:07, 28997.33it/s]

  4%|█████▉                                                                                                                                         | 8708/208923 [00:00<00:06, 28947.74it/s]

  6%|███████▉                                                                                                                                      | 11603/208923 [00:00<00:06, 28493.89it/s]

...

 95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████       | 198578/208923 [00:07<00:00, 27345.21it/s]

 96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉     | 201390/208923 [00:07<00:00, 27571.09it/s]

 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊   | 204201/208923 [00:07<00:00, 27729.41it/s]

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 206977/208923 [00:07<00:00, 27288.78it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 208923/208923 [00:07<00:00, 26734.08it/s]

In [11]:
train_X, train_w = train_ds[:]
wt_enc = train_X[0].argmax(dim=-1)

In [14]:
##############################################################
############## Train Potts Model #############################
##############################################################
device='cuda'

seq_len = len(msa_sequences[0])
num_tokens = 21
model = Potts(seq_len, num_tokens, wt_enc=None)
lam=1e-7
lr=1e-1
n_epoch=20
model = train_sgd(model, train_X, train_w, lam, bs=1000, lr=lr, device=device, n_epoch=n_epoch, mut_seqs_onehot=mut_seqs_onehot, y_dms=y_dms)

  0%|                                                                                                                                                                 | 0/20 [00:00<?, ?it/s]

  0%|                                                                                           | 0/20 [00:05<?, ?it/s, epoch_l=4.33e+7, loss=257, mem=6.2e+8, num_iters=208, spearman=0.682]

  5%|████▏                                                                              | 1/20 [00:05<01:35,  5.04s/it, epoch_l=4.33e+7, loss=257, mem=6.2e+8, num_iters=208, spearman=0.682]

  5%|████▏                                                                              | 1/20 [00:09<01:35,  5.04s/it, epoch_l=3.67e+7, loss=236, mem=6.2e+8, num_iters=416, spearman=0.694]

 10%|████████▎                                                                          | 2/20 [00:09<01:25,  4.75s/it, epoch_l=3.67e+7, loss=236, mem=6.2e+8, num_iters=416, spearman=0.694]

...

 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [01:27<00:09,  4.63s/it, epoch_l=2.61e+7, loss=196, mem=6.2e+8, num_iters=3952, spearman=0.71]

 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [01:27<00:04,  4.65s/it, epoch_l=2.61e+7, loss=196, mem=6.2e+8, num_iters=3952, spearman=0.71]

 95%|██████████████████████████████████████████████████████████████████████████████▊    | 19/20 [01:32<00:04,  4.65s/it, epoch_l=2.6e+7, loss=198, mem=6.2e+8, num_iters=4160, spearman=0.71]

100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [01:32<00:00,  4.65s/it, epoch_l=2.6e+7, loss=198, mem=6.2e+8, num_iters=4160, spearman=0.71]

100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [01:32<00:00,  4.62s/it, epoch_l=2.6e+7, loss=198, mem=6.2e+8, num_iters=4160, spearman=0.71]

In [17]:
def sample_potts(model, replay_buffer, proposal_dist, n_steps, device):
    # get samples from replay buffer
    samples = replay_buffer.sample()
    samples = samples.to(device)
    # update samples with MCMC using proposal_dist
    samples_new = proposal_dist.sample_nsteps(samples, model, n_steps)
    # update replay buffer
    replay_buffer.update(samples_new)
    return samples_new


bs = 1000
proposal_dist = PottsGWGCategoricalSampler(bs, seq_len, num_tokens, device)
init_X = train_X[:bs].to(device)

In [25]:
n_steps = 100
init_X = train_X[:bs].to(device)
samples_new = proposal_dist.sample_nsteps(init_X, model, n_steps)

In [30]:
with torch.no_grad():
    energies = model(samples_new)
    print(energies.mean())
    

tensor(-695.5458, device='cuda:0')


In [31]:
n_steps = 100
init_X = train_X[:bs].to(device)

model.temp = 0.2 # set temperature lower
samples_new = proposal_dist.sample_nsteps(init_X, model, n_steps)

with torch.no_grad():
    energies = model(samples_new)
    print(energies.mean())

# we will get lower energy sample

tensor(-863.5359, device='cuda:0')
